Given an integer array nums, handle multiple queries of the following types:

    Update the value of an element in nums.
    Calculate the sum of the elements of nums between indices left and right inclusive where left <= right.

Implement the NumArray class:

    NumArray(int[] nums) Initializes the object with the integer array nums.
    void update(int index, int val) Updates the value of nums[index] to be val.
    int sumRange(int left, int right) Returns the sum of the elements of nums between indices left and right inclusive (i.e. nums[left] + nums[left + 1] + ... + nums[right]).

 

Example 1:

Input
["NumArray", "sumRange", "update", "sumRange"]
[[[1, 3, 5]], [0, 2], [1, 2], [0, 2]]
Output
[null, 9, null, 8]

Explanation
NumArray numArray = new NumArray([1, 3, 5]);
numArray.sumRange(0, 2); // return 1 + 3 + 5 = 9
numArray.update(1, 2);   // nums = [1, 2, 5]
numArray.sumRange(0, 2); // return 1 + 2 + 5 = 8

 

Constraints:

    1 <= nums.length <= 3 * 104
    -100 <= nums[i] <= 100
    0 <= index < nums.length
    -100 <= val <= 100
    0 <= left <= right < nums.length
    At most 3 * 104 calls will be made to update and sumRange.



In [ ]:
from typing import List


class NumArray:

    def __init__(self, nums: List[int]):
        self.nums = nums
        

    def update(self, index: int, val: int) -> None:
        self.nums[index] = val
        

    def sumRange(self, left: int, right: int) -> int:
        return sum(self.nums[left:right + 1])


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(index,val)
# param_2 = obj.sumRange(left,right)

In [ ]:
from typing import List


class NumArray:

    def __init__(self, nums: List[int]):
        self.nums = nums
        self.prefix_sums = [0]
        for num in nums:
            self.prefix_sums.append(self.prefix_sums[-1] + num)
        

    def update(self, index: int, val: int) -> None:
        old_val = self.nums[index]
        self.nums[index] = val
        diff = val - old_val
        if diff != 0:
            for i in range(index + 1, len(self.prefix_sums)):
                self.prefix_sums[i] += diff
        

    def sumRange(self, left: int, right: int) -> int:
        return self.prefix_sums[right + 1] - self.prefix_sums[left]


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(index,val)
# param_2 = obj.sumRange(left,right)

In [ ]:
# The definition of segment node
class SegmentNode:
    def __init__(self, start, end):

        # The range of sum
        self.start, self.end = start, end

        # Pointers to the left and right child
        self.left, self.right = None, None

        # The total sum over those range
        self.val = 0


class NumArray:
    def __init__(self, nums: list[int]):
  
        # Recursively build a segment tree from a given list of numbers
        def createTree(l, r):

            # If the left and right pointers meet, this is the leaf of the tree 
            if l == r:
                n = SegmentNode(l, r)
                n.val = nums[l]
                return n

            # Else, calculate the mid pointer
            mid = (r - l) // 2 + l

            # Create a sigment node
            node = SegmentNode(l, r)

            # Recursively build the left child of this segment node
            node.left = createTree(l, mid)

            # Recursively build the right child of this segment node
            node.right = createTree(mid + 1, r)

            # Update the sum of this node with the sum of the left child and the sum of the right child
            node.val = node.left.val + node.right.val

            # Return the current node
            return node

        # Save nums such that we can quickly update nodes
        self.nums = nums

        # Build the segment tree
        self.root = createTree(0, len(nums) - 1)

    def update(self, index: int, val: int) -> None:

        # Calculate the difference that needed to be add all all nodes
        diff = val - self.nums[index]

        # Update the value in nums at index
        self.nums[index] = val

        # Recursively update all nodes
        def updateVal(node):

            # Add the difference to the current node
            node.val += diff

            # If there is a left child and the index fall between its range, recursively update the left child
            if node.left and node.left.start <= index <= node.left.end:
                updateVal(node.left)
            
            # Else if there is a right child and the index falls between its range, recursively update the right child
            elif node.right and node.right.start <= index <= node.right.end:
                updateVal(node.right)

        # Recursively update all nodes from the root
        updateVal(self.root)

    def sumRange(self, left: int, right: int) -> int:

        # Recursively calculate the sum over a given range
        def rangeSum(node, l, r):

            # If the range of the current node matches the left and right pointers, return the sum of this node
            if node.start == l and node.end == r:
                return node.val

            # Else, calculate the mid pointer
            mid = (node.end - node.start) // 2 + node.start

            # If the mid pointer falls between the left and right pointers, calculate the range sum by summing the partial sum from both children
            if l <= mid < r:
                return rangeSum(node.left, l, mid) + rangeSum(node.right, mid + 1, r)

            # If both pointers are less than the mid pointers, calculate the range sum from the left child
            if l <= r <= mid:
                return rangeSum(node.left, l, r)

            # Else, calculate the range sum from the right child
            return rangeSum(node.right, l, r)

        # Recursively calculate the range sum starting from the root
        return rangeSum(self.root, left, right)	

In [ ]:
class NumArray(object):
    # Build segment tree
    # For example if nums = [1, 3, 5]
    # We segment tree is initialized to be double of its length:
    # [0, 0, 0, 1, 3, 5]
    # Then we compute range sums by tree[i] = tree[2*i] + tree[2*i+1], up to i = 1
    # [0, 9, 8, 1, 3, 5]
    # To update, we go to the index, update it, and trace back to update all the involved range sum
    # Given two indices to find the inclusive range sum, because the precalculated range sums is defined to be tree[2*i] + tree[2*i+1], we add tree[i] if i is odd, increment i and take i // 2. Also we add trees[j] and decrement j by 1 if j is even, and take j // 2
    # we stop when i > j
    
    def __init__(self, nums):
        self.n = len(nums)
        self.tree = [0] * (2 * self.n)
        for i, num in enumerate(nums):
            self.tree[self.n + i] = num
        for i in range(self.n - 1, 0, -1):
            self.tree[i] = self.tree[2*i] + self.tree[2*i+1]
        #print(self.tree)

    def update(self, i, val):
        i += self.n
        self.tree[i] = val
        i //= 2
        while i > 0:
            self.tree[i] = self.tree[2*i] + self.tree[2*i + 1]
            i //= 2
        #print(self.tree)

    def sumRange(self, i, j):
        i += self.n
        j += self.n
        s = 0
        while i <= j:
            #print(i,j)
            if i % 2 == 1:
                s += self.tree[i]
                i += 1
            if j % 2 == 0:
                s += self.tree[j]
                j -= 1
            i //= 2
            j //= 2
        return s

# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(i,val)
# param_2 = obj.sumRange(i,j)


In [ ]:
class NumArray:

    def __init__(self, nums: List[int]):
        self.sqrt = int(len(nums)**(0.5))
        self.sumarr = [0]*(len(nums)//self.sqrt+(len(nums)%self.sqrt!=0)*1)
        self.nums = nums
        for i in range(len(nums)):
            self.sumarr[i//self.sqrt]+=nums[i]
        
        
    def update(self, index: int, val: int) -> None:
        oldv = self.nums[index]
        self.nums[index] = val
        self.sumarr[index//self.sqrt] += val - oldv 

    def sumRange(self, left: int, right: int) -> int:
        lidx = left//self.sqrt
        ridx = right//self.sqrt
        tsum = 0
        
        for i in range(lidx+1,ridx): # Overlapping blocks
            tsum+=self.sumarr[i]
            
        for i in range(left,min(right+1,((lidx+1)*self.sqrt))): # first block
            tsum+=self.nums[i]
                       
        if ridx!=lidx: # last block
            for i in range(ridx*self.sqrt,right+1):
                tsum+=self.nums[i]
        return tsum


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(index,val)
# param_2 = obj.sumRange(left,right)
